In [7]:
import pandas as pd
import random
import os
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename

app = Flask(__name__)

font_path = r'D:\AI(A)\fonts\NanumBarunGothic.ttf'

skincare_data  = pd.read_csv('skincare3.csv')
font_prop = fm.FontProperties(fname=font_path)

class SkincareProductRecommender:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def check_ingredient_presence(self, product_name, ingredient_list):
        product = self.dataframe[self.dataframe['name'] == product_name]
        if not product.empty:
            ingredients_list = eval(product['elements'].values[0])
            # Check for both full and partial matches within each ingredient
            return any(any(ingredient_part in ingredient for ingredient_part in ingredient_list) for ingredient in ingredients_list)
        return False

    def recommend_products_excluding_bad_combinations(self, good_combination, bad_combination):
        recommended_products = []
        for index, row in self.dataframe.iterrows():
            ingredients_list = eval(row['elements'])
            contains_bad = any(bad in ingredient for ingredient in ingredients_list for bad in bad_combination)
            contains_good = any(good in ingredient for ingredient in ingredients_list for good in good_combination)
            if contains_good and not contains_bad:
                recommended_products.append(row['name'])
        return recommended_products

    def categorize_and_recommend_products(self, recommended_product_names):
        categorized_recommendations = {'에센스': [], '스킨': [], '크림': [], '로션': []}
        for product_name in recommended_product_names:
            product_row = self.dataframe[self.dataframe['name'] == product_name]
            if not product_row.empty:
                for category in categorized_recommendations:
                    if product_row[category].values[0] == 1:
                        categorized_recommendations[category].append(product_name)

        # Select a random product from each category
        for category in categorized_recommendations:
            if categorized_recommendations[category]:
                categorized_recommendations[category] = [random.choice(categorized_recommendations[category])]

        return categorized_recommendations

    
    def check_and_recommend(self, product_name, ingredient_check, good_combination, bad_combination):
        if self.check_ingredient_presence(product_name, ingredient_check):
            recommended_products = self.recommend_products_excluding_bad_combinations(good_combination, bad_combination)
            return self.categorize_and_recommend_products(recommended_products)
        return {}


@app.route('/submit', methods=['POST'])
def submit():
    text = request.form['txta']
    print(text)

    product_name_to_check=text
    
    skincare_data = pd.read_csv('skincare3.csv')  # CSV 파일 경로 확인 필요
    recommender = SkincareProductRecommender(skincare_data)

    vitamin_c_ingredients = ["비타민C", "아스코빅애씨드", "에칠아스코빌에텔", "아스코빅애씨드폴리펩타이드", "아스코빌글루코사이드"]
    good_combination_ingredients = ["펩타이드", "매트릭실"]
    bad_combination_ingredients = ["콜라겐", "알파하이드록시산", "살리실릭애씨드", "실리실산"]

    retinol_ingredients = [
        "레티놀", "레티닐팔미테이트", "레티닐아세테이트", "레티닐프로피오네이트", 
        "레티닐리놀리에이트", "폴리에톡시레이티드레틴아마이드", "레티닐레티노에이트", "레틴알", 
        "레티노익애씨드", "레티노익애씨드 유도체", "트레티노인"
    ]
    good_combinations_with_retinol = [
        "펩타이드", "매트릭실", "히알루론산", "하이드롤라이즈드 히알루론산", 
        "소듐 히알루로네이트", "하이알루"
    ]
    bad_combinations_with_retinol = ["알파하이드록시산", "아젤틱애씨드", "트로첼로로아세틱애씨드", "살리실산", "벤조일페놀"]

    niacinamide_ingredients = ["나이아신아마이드"]
    good_combinations_with_niacinamide = [
        "알부틴", "아스코빌글루코사이드", "에칠아스코빌에텔", "아스코빌테트라이소팔미테이트",
        "실리실산", "실리실릭애씨드", "히얄루론산", "세콜지", "판테놀"
    ]
    bad_combinations_with_niacinamide = [ ]


    vitamin_c_recommendations = recommender.check_and_recommend(
        product_name_to_check,
        vitamin_c_ingredients,
        good_combination_ingredients,
        bad_combination_ingredients
    )

    retinol_recommendations = recommender.check_and_recommend(
        product_name_to_check,
        retinol_ingredients,
        good_combinations_with_retinol,
        bad_combinations_with_retinol
    )

    niacinamide_recommendations = recommender.check_and_recommend(
        product_name_to_check,
        niacinamide_ingredients ,
        good_combinations_with_niacinamide,
        bad_combinations_with_niacinamide
    )


    vitamin_c_present = recommender.check_ingredient_presence(product_name_to_check, vitamin_c_ingredients)
    retinol_present = recommender.check_ingredient_presence(product_name_to_check, retinol_ingredients)
    niacinamide_present = recommender.check_ingredient_presence(product_name_to_check, niacinamide_ingredients)


    arr = []
    for category, product_names in niacinamide_recommendations.items():
        if product_names:
            print(product_names[0])
            arr.append(product_names[0])
    data = {
        'essence':arr[0],
        'skin': arr[1],
        'crime' : arr[2],
        'location': arr[3] }
    print(data)

    return data
if __name__ == '__main__':
    app.run(host='0.0.0.0')



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.40:5000
Press CTRL+C to quit


[2023-11-24 09:42:34,848] ERROR in app: Exception on /submit [POST]
Traceback (most recent call last):
  File "c:\Users\user\miniconda3\envs\final\lib\site-packages\flask\app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\user\miniconda3\envs\final\lib\site-packages\flask\app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\user\miniconda3\envs\final\lib\site-packages\flask\app.py", line 867, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\user\miniconda3\envs\final\lib\site-packages\flask\app.py", line 852, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_15708\1951698925.py", line 136, in submit
    'essence':arr[0],
IndexError: list index out of range
192.168.0.38 - - [24/Nov/2023 09:42:34] "POST /submit HTTP/1.1" 500 -


asfasfafs


192.168.0.38 - - [24/Nov/2023 09:43:50] "POST /submit HTTP/1.1" 200 -


피지오겔 DMT포맨
코스알엑스 더 비타민C23 세럼 더블기획
아누아 어성초 77 수딩 토너
라운드랩 자작나무 수분 크림
라운드랩 1025 독도 로션
{'essence': '코스알엑스 더 비타민C23 세럼 더블기획', 'skin': '아누아 어성초 77 수딩 토너', 'crime': '라운드랩 자작나무 수분 크림', 'location': '라운드랩 1025 독도 로션'}


192.168.0.38 - - [24/Nov/2023 09:44:51] "POST /submit HTTP/1.1" 200 -


피지오겔 DMT포맨
피지오겔 사이언수티컬즈 데일리뮨 앰플 세럼
넘버즈인 1번 진정 맑게담은 청초토너
라운드랩 자작나무 수분 크림
라운드랩 1025 독도 로션
{'essence': '피지오겔 사이언수티컬즈 데일리뮨 앰플 세럼', 'skin': '넘버즈인 1번 진정 맑게담은 청초토너', 'crime': '라운드랩 자작나무 수분 크림', 'location': '라운드랩 1025 독도 로션'}


192.168.0.38 - - [24/Nov/2023 09:45:11] "POST /submit HTTP/1.1" 200 -


피지오겔 DMT포맨
메디필 히알루론산 레이어 물톡스 앰플 더블
브링그린 티트리시카수딩토너
이즈앤트리 어니언 뉴페어 겔크림
라운드랩 1025 독도 로션
{'essence': '메디필 히알루론산 레이어 물톡스 앰플 더블', 'skin': '브링그린 티트리시카수딩토너', 'crime': '이즈앤트리 어니언 뉴페어 겔크림', 'location': '라운드랩 1025 독도 로션'}
